# Step 0: Setup

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras

## Step 1: Preprocess the Data
Preprocess the dataset using Pandas and `StandardScaler` from scikit-learn.

In [2]:
# Read in `charity_data.csv` to a Pandas DataFrame.
raw_df = pd.read_csv("../../Resources/charity_data.csv")

*What variable(s) are the target(s) for the model?* 
'IS_SUCCESSFUL'

*What variable(s) are the feature(s) for the model?*
'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'

In [3]:
# Drop the 'EIN' and 'NAME' columns.
dropped_df = raw_df.drop(columns=['EIN', 'NAME'])

For columns that have more than 10 unique values, determine the number of data points for each unique value.

In [4]:
# Look at 'APPLICATION_TYPE' value counts for binning.
app_type_counts = dropped_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at 'CLASSIFICATION' value counts for binning.
classification_counts = dropped_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts > 1.
classification_counts_GT1 = dropped_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]
classification_counts_GT1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

Use the number of data points for each unique value to pick a cutoff point to bin "rare" categorical variables together in a new value, `Other`, and then check if the binning was successful.

In [7]:
# Choose a cutoff value and create a list of application types to be replaced.
application_types_to_replace = app_type_counts.loc[lambda x : x < 100].index

# Replace in DataFrame.
for app in application_types_to_replace:
    dropped_df['APPLICATION_TYPE'] = dropped_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful.
dropped_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced.
classifications_to_replace = classification_counts.loc[lambda x : x < 150].index

# Replace in DataFrame.
for cls in classifications_to_replace:
    dropped_df['CLASSIFICATION'] = dropped_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful.
dropped_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric (i.e., encode categorial variables) with `pd.get_dummies`.
df = pd.get_dummies(dropped_df)
df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


Split the preprocessed data into a features array, `X`, and a target array, `y`. Use these arrays and the `train_test_split` function to split the data into training and testing datasets.

In [10]:
# Split our preprocessed data into our features and target arrays.
X = df.drop(columns='IS_SUCCESSFUL')
y = df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Scale the training and testing features datasets by creating a `StandardScaler` instance, fitting it to the training data, then using the `transform` function.

In [11]:
# Create a StandardScaler instances.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Step 2: Compile, Train and Evaluate the Model
Design a neural network / deep learning model to create a binary classification model that can predict if an Alphabet Soup-funded organization will be successful based on the features in the dataset. Consider how many inputs there are before determining the number of neurons and layers in your model. Then compile, train, and evaluate the binary classification model to calculate the model’s loss and accuracy.

In [12]:
# Define the model - deep neural network, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 30
hidden_nodes_layer4 = 15

nn = tf.keras.models.Sequential()

# Create the first hidden layer and choose an appropriate activation function.
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))
# Add a second hidden layer with an appropriate activation function.
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu', input_dim=number_input_features))
# Add a third hidden layer with an appropriate activation function.
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu', input_dim=number_input_features))
# Add a fourth hidden layer with an appropriate activation function.
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation='relu', input_dim=number_input_features))
# Create an output layer with an appropriate activation function.
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model.
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1440      
                                                                 
 dense_1 (Dense)             (None, 30)                930       
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 dense_3 (Dense)             (None, 15)                465       
                                                                 
 dense_4 (Dense)             (None, 1)                 16        
                                                                 
Total params: 3,781
Trainable params: 3,781
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model.
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model. Create a callback that saves the model's weights every five epochs.
mc = keras.callbacks.ModelCheckpoint('Weights/weights{epoch:08d}.h5', save_weights_only=True, period=5)
fit = nn.fit(X_train_scaled, y_train, epochs=200, callbacks=[mc])

Epoch 1/200
804/804 [==============================] - 4s 3ms/step - loss: 0.5721 - accuracy: 0.7164
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5537 - accuracy: 0.7300
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5504 - accuracy: 0.7320
Epoch 4/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5491 - accuracy: 0.7319
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5475 - accuracy: 0.7328
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5462 - accuracy: 0.7346
Epoch 7/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5455 - accuracy: 0.7343
Epoch 8/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5452 - accuracy: 0.7332
Epoch 9/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5443 - accuracy: 0.7330
Epoch 10/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5444 - accura

804/804 [==============================] - 3s 3ms/step - loss: 0.5334 - accuracy: 0.7400
Epoch 81/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5333 - accuracy: 0.7406
Epoch 82/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5327 - accuracy: 0.7411
Epoch 83/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5334 - accuracy: 0.7405
Epoch 84/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5330 - accuracy: 0.7408
Epoch 85/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5333 - accuracy: 0.7403
Epoch 86/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5333 - accuracy: 0.7402
Epoch 87/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5331 - accuracy: 0.7411
Epoch 88/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5332 - accuracy: 0.7411
Epoch 89/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5332 - accuracy: 

804/804 [==============================] - 3s 3ms/step - loss: 0.5304 - accuracy: 0.7418
Epoch 160/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5303 - accuracy: 0.7411
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5304 - accuracy: 0.7414
Epoch 162/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5304 - accuracy: 0.7419
Epoch 163/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5305 - accuracy: 0.7416
Epoch 164/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5300 - accuracy: 0.7413
Epoch 165/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5303 - accuracy: 0.7418
Epoch 166/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5305 - accuracy: 0.7416
Epoch 167/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5301 - accuracy: 0.7417
Epoch 168/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5302 - a

In [15]:
# Evaluate the model using the test data.
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5753 - accuracy: 0.7296 - 907ms/epoch - 3ms/step
Loss: 0.575302004814148, Accuracy: 0.7295626997947693


In [16]:
# Save and export results to an HDF5 file.
nn.save('Models/AlphabetSoupCharity_Optimization.h5')